In [1]:
import numpy as np
import nltk
import random
import string
import sklearn
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\19928\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
import pandas
faq = pandas.read_csv("Service.txt", delimiter="\t", header=None, names=["Q", "A"], encoding='iso-8859-1')
faq.shape

(6, 2)

In [10]:
faq

,Q,A
0,What is the theory behind the recommendaton?,"Item-based recommendation theory is employed, ..."
1,How to use the game recommendation system?,Simply input your favourite game or preferred ...
2,How many games are included in recommendation ...,"There are more than 18,000 games pulished betw..."
3,Can you recommend some good games to me?,"Sure, games like Starfield, Chants of Sennaar ..."
4,Where can I download the recommended games?,All the games are available to download in Ste...
5,Why does the system go wrong?,"Apologies, Maybe you could check if the App is..."


In [11]:
qns = faq["Q"]
answers = faq["A"]
TfidfVec = TfidfVectorizer()
tfidf = TfidfVec.fit_transform(qns)
tfidf.shape

(6, 30)

In [17]:
#the basic chatbot
#prepare some greeting words
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)


# function to match input to the preprocessed sentences
def response(user_response):
    robo_response = ''
    Q = ''
    A = ''
    new = TfidfVec.transform([user_response]) #vectorize the input to the same dimension space
    vals = cosine_similarity(new[0], tfidf)
    flat = vals.flatten()
    idx = flat.argsort()[-1]
    sim_max = flat[idx]
    if(sim_max<=0.2):
        robo_response = "I am sorry! I don't have answer for that."
        return robo_response, Q, sim_max, A
       # return robo_response
    else:
        robo_response = "Similar question found!"
        Q = qns[idx]
        A = "Ans: "+answers[idx]
        return robo_response, Q, sim_max, A
        #return robo_response, A

def say(robo_response, Q, score, A):
  #print(robo_response)
  #print(Q + ' (' + str(score) + ")")
  print(A)
  return A


In [19]:
say(*response("Can you give me some recommendations?"))

Ans: Sure, games like Starfield, Chants of Sennaar and Citizen Sleeper: Purge have high rates which suggests they are worth playing. For more personalized recommendation, please enter your favourite game or preferred genre.


'Ans: Sure, games like Starfield, Chants of Sennaar and Citizen Sleeper: Purge have high rates which suggests they are worth playing. For more personalized recommendation, please enter your favourite game or preferred genre.'

In [20]:
#starting the bot
flag=True
print("Bill: My name is Bill. I am the Service bot for this recommendation system, if you have any questions, please feel free to ask. If you want to end the conversation, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response.lower() !='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("Bill: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("Bill: "+greeting(user_response))
            else:
                print("Bill: ",end="")
                say(*response(user_response))
    else:
        flag=False
        print("Bill: Bye! take care...")

Bill: My name is Bill. I am the Service bot for this recommendation system, if you have any questions, please feel free to ask. If you want to end the conversation, type Bye!
Can you give me some recommendations
Bill: Ans: Sure, games like Starfield, Chants of Sennaar and Citizen Sleeper: Purge have high rates which suggests they are worth playing. For more personalized recommendation, please enter your favourite game or preferred genre.
bye
Bill: Bye! take care...
